# Ejemplo de Red Neuronal con Aprendizaje No Supervisado

Para instalar los drivers de nvidia
[Nvidia Developer](https://developer.nvidia.com/cuda-downloads?target_os=Windows)

In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras.models import Model
from sklearn.cluster import KMeans
from sklearn.datasets import make_blobs

2024-12-26 19:40:05.636161: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-26 19:40:05.649482: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735260005.666543  710969 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735260005.671571  710969 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-12-26 19:40:05.688654: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

In [2]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    tf.config.experimental.set_virtual_device_configuration(
        gpus[0],[tf.config.experimental.VirtualDeviceConfiguration(memory_limit=5120)])
  except RuntimeError as e:
    print(e)

2024-12-26 19:40:09.113365: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:152] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: CUDA_ERROR_COMPAT_NOT_SUPPORTED_ON_DEVICE: forward compatibility was attempted on non supported HW
2024-12-26 19:40:09.113430: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:137] retrieving CUDA diagnostic information for host: ragnarok
2024-12-26 19:40:09.113438: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:144] hostname: ragnarok
2024-12-26 19:40:09.113542: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:168] libcuda reported version is: 560.35.5
2024-12-26 19:40:09.113565: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:172] kernel reported version is: 535.183.1
2024-12-26 19:40:09.113576: E external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:262] kernel version 535.183.1 does not match DSO version 560.35.5 -- cannot find working devices

In [3]:
# Generar datos no etiquetados
data, _ = make_blobs(n_samples=1000, centers=3, n_features=10, random_state=42)
data

array([[-10.86666274,   8.50276908,   5.31228432, ...,   0.62908701,
         -1.45621127,  -4.50845482],
       [ -9.33574127,  10.78272909,   7.09305114, ...,   0.39721339,
          0.91133509,  -3.28638007],
       [ -3.93100829,  10.51561978,   4.317199  , ...,   9.48852526,
          1.378782  ,   5.08929168],
       ...,
       [  2.15396857,  -6.56124086,  -4.74150801, ...,   1.64398637,
          1.70115187,  -9.72761582],
       [  2.86473831,  -5.54548096,  -3.89335144, ...,  -1.23520403,
          2.77454887,  -9.45699432],
       [  1.26423   ,  -7.2476863 ,  -3.58734811, ...,  -0.09493573,
          2.70544977,  -7.54635315]])

In [4]:
# Construcción del autoencoder
input_dim = data.shape[1]
encoding_dim = 2  # Dimensión reducida

input_layer = Input(shape=(input_dim,))
encoded = Dense(encoding_dim, activation='relu')(input_layer)
decoded = Dense(input_dim, activation='sigmoid')(encoded)

autoencoder = Model(inputs=input_layer, outputs=decoded)
encoder = Model(inputs=input_layer, outputs=encoded)

In [5]:
autoencoder.compile(optimizer='adam', loss='mse')
autoencoder.fit(data, data, epochs=50, batch_size=8, verbose=0)

In [6]:
# Extracción de características reducidas
reduced_data = encoder.predict(data)

32/32 ━━━━━━━━━━━━━━━━━━━━ 0s 820us/step


In [7]:
# Agrupamiento con KMeans
kmeans = KMeans(n_clusters=3)
clusters = kmeans.fit_predict(reduced_data)

print("Etiquetas de los clústeres:", clusters)


Etiquetas de los clústeres: [1 1 2 2 1 0 1 2 0 1 2 1 2 1 0 1 2 2 0 2 0 2 0 1 2 1 0 2 1 2 0 0 1 0 2 2 1
 1 2 0 1 0 2 0 2 2 0 2 2 2 2 2 0 2 0 1 0 2 2 1 1 2 1 0 0 0 1 2 0 0 1 1 2 2
 0 1 1 1 0 2 1 2 1 0 1 0 1 1 0 1 2 1 2 2 2 2 2 1 2 0 2 0 2 2 0 2 2 0 2 0 2
 1 2 2 0 0 0 0 0 0 1 2 1 0 1 0 0 1 2 1 2 2 2 1 0 0 0 1 1 1 2 1 1 1 1 0 1 0
 0 0 1 1 1 0 1 1 0 0 2 2 1 2 1 0 1 0 2 2 1 1 1 1 0 0 1 2 1 2 2 1 2 2 1 2 2
 1 1 0 0 0 1 1 0 2 1 0 0 0 1 2 0 1 2 1 2 2 0 2 2 2 1 0 2 2 2 0 0 0 2 0 0 0
 0 2 1 1 1 0 1 2 0 2 1 1 0 1 1 2 1 2 2 2 0 0 1 0 2 1 0 2 1 0 0 0 2 0 2 1 1
 2 1 0 1 1 1 2 0 0 2 1 0 0 0 1 2 2 2 0 0 0 2 0 1 0 2 1 2 1 2 2 1 2 0 2 1 1
 2 2 0 1 0 2 0 1 2 2 1 0 0 2 0 1 1 2 2 2 0 0 1 0 1 1 1 0 0 1 0 2 2 1 2 1 0
 0 2 0 2 2 0 2 1 1 0 0 2 2 1 0 0 1 1 1 1 0 2 0 2 1 2 2 0 1 1 1 2 0 1 1 1 2
 2 2 2 1 1 1 2 0 2 2 0 2 2 1 1 1 2 1 0 0 2 0 1 2 0 0 0 2 0 0 2 2 1 1 2 1 0
 1 1 1 0 0 0 0 2 0 1 0 1 2 2 2 0 0 1 1 2 1 0 1 1 0 2 0 0 1 1 0 0 0 0 1 0 1
 1 1 1 0 0 0 2 2 0 0 0 0 2 1 2 0 1 2 2 1 1 0 0 0 2 1 1 1 2 1 1 1 0 0 2 2